In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pylab as plt
import torch
import numpy as np
import seaborn as sn
sn.set_context("poster")
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torchvision import transforms, datasets

from deepsith import DeepSITH

import numpy as np
import scipy
import scipy.stats as st
import scipy.special
import scipy.signal
import scipy.interpolate

import pandas as pd

from os.path import join
import random
from csv import DictWriter

from tqdm.notebook import tqdm
import pickle
# if gpu is to be used
use_cuda = torch.cuda.is_available()
print(use_cuda)
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
ttype = FloatTensor

In [ ]:
import torch
import numpy as np

def get_batch(batch_size, T, ttype):
    values = torch.rand(T, batch_size, requires_grad=False)
    indices = torch.zeros_like(values)
    half = int(T / 2)
    for i in range(batch_size):
        half_1 = np.random.randint(half)
        hals_2 = np.random.randint(half, T)
        indices[half_1, i] = 1
        indices[hals_2, i] = 1

    data = torch.stack((values, indices), dim=-1).type(ttype)
    targets = torch.mul(values, indices).sum(dim=0).type(ttype)
    return data, targets


In [ ]:
torch.manual_seed(1111)

In [ ]:
def train(model, ttype, seq_length, optimizer, loss_func, 
          epoch, perf_file, loss_buffer_size=20, batch_size=1, test_size=10,
          device='cuda', prog_bar=None):
    assert(loss_buffer_size%batch_size==0)

    losses = []
    perfs = []
    last_test_perf = 0
    for batch_idx in range(20000):
        model.train()
        sig, target = get_batch(batch_size, seq_length, ttype=ttype)
        sig = sig.transpose(0,1).transpose(1,2).unsqueeze(1)
        target = target.unsqueeze(1)
        optimizer.zero_grad()
        out = model(sig)
        loss = loss_func(out[:, -1, :],
                         target)
         
        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().numpy())
        losses = losses[-loss_buffer_size:]
        if not (prog_bar is None):
            # Update progress_bar
            s = "{}:{} Loss: {:.8f}"
            format_list = [e, int(batch_idx/(50/batch_size)), np.mean(losses)]         
            s = s.format(*format_list)
            prog_bar.set_description(s)
        if ((batch_idx*batch_size)%loss_buffer_size == 0) & (batch_idx != 0):
            loss_track = {}
            #last_test_perf = test_norm(model, 'cuda', test_sig, test_class,
            #                                    batch_size=test_size, 
            #                                    )
            loss_track['avg_loss'] = np.mean(losses)
            #loss_track['last_test'] = last_test_perf
            loss_track['epoch'] = epoch
            loss_track['batch_idx'] = batch_idx
            with open(perf_file, 'a+') as fp:
                csv_writer = DictWriter(fp, fieldnames=list(loss_track.keys()))
                if fp.tell() == 0:
                    csv_writer.writeheader()
                csv_writer.writerow(loss_track)
                fp.flush()
def test_norm(model, device, seq_length, loss_func, batch_size=100):
    model.eval()
    correct = 0
    count = 0
    with torch.no_grad():
        sig, target = get_batch(batch_size, seq_length, ttype=ttype)
        sig = sig.transpose(0,1).transpose(1,2).unsqueeze(1)
        target = target.unsqueeze(1)
        out = model(sig)
        loss = loss_func(out[:, -1, :],
                         target)
    return loss

In [ ]:
class DeepSITH_Predictor(nn.Module):
    def __init__(self, out_features, layer_params, dropout=.5):
        super(DeepSITH_Predictor, self).__init__()
        last_hidden = layer_params[-1]['hidden_size']
        self.hs = DeepSITH(layer_params=layer_params, dropout=dropout)
        self.to_out = nn.Linear(last_hidden, out_features)
    def forward(self, inp):
        x = self.hs(inp)
        x = self.to_out(x)
        return x

In [ ]:
import scipy.optimize as opt
from deepsith import iSITH
def min_fun(x, *args):
    ntau = int(args[2])
    k = int(x[0])
    if k < 4 or k>125:
        return np.inf
    tau_min = args[0]
    tau_max = args[1]    
    ev = iSITH(tau_min=tau_min, tau_max=tau_max, buff_max=tau_max*5, k=k, ntau=ntau, dt=1, g=1.0)    
    std_0 = ev.filters[:, 0, 0, :].detach().cpu().T.numpy()[::-1].sum(1)[int(tau_min):int(tau_max)].std()
    std_1 = ev.filters[:, 0, 0, :].detach().cpu().T.numpy()[::-1, ::2].sum(1)[int(tau_min):int(tau_max)].std()    
    to_min = std_0/std_1
    return to_min

# T = 100

In [ ]:
g = 0.0
sith_params1g = {"in_features":2, 
                "tau_min":1, "tau_max":20.0, 'buff_max':30, 
                "k":74, 'dt':1,
                "ntau":13, 'g':g,  
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params2g = {"in_features":sith_params1g['hidden_size'], 
                "tau_min":1, "tau_max":120.0, 'buff_max':250, 
                "k":27, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params3g = {"in_features":sith_params2g['hidden_size'], 
                "tau_min":1, "tau_max":720.0, 'buff_max':2000, 
                "k":14, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params4g = {"in_features":sith_params3g['hidden_size'], 
                "tau_min":1, "tau_max":4320.0, 'buff_max':12000, 
                "k":8, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}

layer_paramsg = [sith_params1g, sith_params2g, sith_params3g, sith_params4g]
model = DeepSITH_Predictor(out_features=1,
                           layer_params=layer_params, 
                           dropout=.0).cuda()

tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)

In [ ]:
seq_length=100

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 1
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
for e in progress_bar:
    train(model, ttype, seq_length,
          optimizer, loss_func, batch_size=50, loss_buffer_size=100,
          epoch=e, perf_file=join('perf','adding100_deepsith_1_same.csv'),
          prog_bar=progress_bar)

# T = 500

In [ ]:
g = 0.0
sith_params1g = {"in_features":2, 
                "tau_min":1, "tau_max":20.0, 'buff_max':30, 
                "k":74, 'dt':1,
                "ntau":13, 'g':g,  
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params2g = {"in_features":sith_params1g['hidden_size'], 
                "tau_min":1, "tau_max":120.0, 'buff_max':250, 
                "k":27, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params3g = {"in_features":sith_params2g['hidden_size'], 
                "tau_min":1, "tau_max":720.0, 'buff_max':2000, 
                "k":14, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params4g = {"in_features":sith_params3g['hidden_size'], 
                "tau_min":1, "tau_max":4320.0, 'buff_max':12000, 
                "k":8, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}

layer_paramsg = [sith_params1g, sith_params2g, sith_params3g, sith_params4g]
model = DeepSITH_Predictor(out_features=1,
                           layer_params=layer_paramsg, 
                           dropout=.0).cuda()

tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)

In [ ]:
seq_length=500

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 1
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
for e in progress_bar:
    train(model, ttype, seq_length,
          optimizer, loss_func, batch_size=50, loss_buffer_size=100,
          epoch=e, perf_file=join('perf','adding500_deepsith_1_same.csv'),
          prog_bar=progress_bar)

# T = 2000

In [ ]:
g = 0.0
sith_params1g = {"in_features":2, 
                "tau_min":1, "tau_max":20.0, 'buff_max':30, 
                "k":74, 'dt':1,
                "ntau":13, 'g':g,  
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params2g = {"in_features":sith_params1g['hidden_size'], 
                "tau_min":1, "tau_max":120.0, 'buff_max':250, 
                "k":27, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params3g = {"in_features":sith_params2g['hidden_size'], 
                "tau_min":1, "tau_max":720.0, 'buff_max':2000, 
                "k":14, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params4g = {"in_features":sith_params3g['hidden_size'], 
                "tau_min":1, "tau_max":4320.0, 'buff_max':12000, 
                "k":8, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}

layer_paramsg = [sith_params1g, sith_params2g, sith_params3g, sith_params4g]
model = DeepSITH_Predictor(out_features=1,
                           layer_params=layer_paramsg, 
                           dropout=.0).cuda()

tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)

In [ ]:
seq_length=2000

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 1
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
for e in progress_bar:
    train(model, ttype, seq_length,
          optimizer, loss_func, batch_size=50, loss_buffer_size=100,
          epoch=e, perf_file=join('perf','adding2000_deepsith_1_same.csv'),
          prog_bar=progress_bar)

#  T = 5000

In [ ]:
g = 0.0
sith_params1g = {"in_features":2, 
                "tau_min":1, "tau_max":20.0, 'buff_max':30, 
                "k":74, 'dt':1,
                "ntau":13, 'g':g,  
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params2g = {"in_features":sith_params1g['hidden_size'], 
                "tau_min":1, "tau_max":120.0, 'buff_max':250, 
                "k":27, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params3g = {"in_features":sith_params2g['hidden_size'], 
                "tau_min":1, "tau_max":720.0, 'buff_max':2000, 
                "k":14, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}
sith_params4g = {"in_features":sith_params3g['hidden_size'], 
                "tau_min":1, "tau_max":4320.0, 'buff_max':12000, 
                "k":8, 'dt':1,
                "ntau":13, 'g':g, 
                "ttype":ttype, 'batch_norm':False,
                "hidden_size":25, "act_func":nn.ReLU()}

layer_paramsg = [sith_params1g, sith_params2g, sith_params3g, sith_params4g]


model = DeepSITH_Predictor(out_features=1,
                           layer_params=layer_paramsg, 
                           dropout=.0).cuda()

tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)

In [ ]:
seq_length=5000

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 1
progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
for e in progress_bar:
    train(model, ttype, seq_length,
          optimizer, loss_func, batch_size=50, loss_buffer_size=100,
          epoch=e, perf_file=join('perf','adding5000_deepsith_1_same.csv'),
          prog_bar=progress_bar)